In [ ]:
import os, time
import torch, numpy
import cv2
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from pathlib import Path

try:
    import segmentation_models_pytorch as smp
except:
    !pip install segmentation-models-pytorch
import segmentation_models_pytorch as smp

In [ ]:
torch.manual_seed(3)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
try:
    import wandb
except:
    !pip install wandb
import wandb

wandb.login(key = "280aa3837eb27ece3c32ed8e27e3e233d0afdc9c")
wandb.init(project="Unet_Deep_Learing_Assignment")

# Config

In [ ]:
config = {
    ### Path bộ data được sample
    "sample_txt": None, # "/kaggle/working/sample_150_data.txt" # nếu train cả bộ data thì để None
    "number_of_sample_data": 150,
    
    ### Path của folder
    "train_folder": "/kaggle/input/bkai-igh-neopolyp/train/train",
    "train_gt_folder": "/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt",
    
    ### Thông số chia tập train - val
    "train_percent": 0.8,
    
    ### Thông số cho quá trình train
    "epochs" : 50,
    "batch_size" : 8,
    "learning_rate" : 0.0001,
    
    ### Load và Save model
    "load_model": None,
    "save_folder": "/kaggle/working/save"
}

# **0. Sample ra bộ data nhỏ hơn để train và đánh giá thử**
Ta sẽ lấy random ra 1 bộ data nhỏ hơn (150 samples) để train và đánh giá thử cho các model.  
Sau đó sẽ chọn ra model tốt nhất rồi train bằng cả bộ data gốc.

In [ ]:
if config["sample_txt"] != None:
    numpy.random.seed(3)
    sample_files = numpy.random.choice(a = os.listdir(config["train_folder"]), size = config["number_of_sample_data"])
    with open(config["sample_txt"], "w") as f:
        for file_name in sample_files:
            f.write(f"{file_name}\n")

# **1 .Preprocess data. Dataset và DataLoader**

In [ ]:
def createPreprocessTransform():
    return transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.Resize((800, 1280)),
        transforms.ToTensor(),
    ])

In [ ]:
class NeopolypDataset(Dataset):
    def __init__(self, 
                 preprocess_transform: transforms.Compose,
                 sample_data_txt: str = config["sample_txt"],
                 train_folder: str = config["train_folder"],
                 train_gt_folder: str = config["train_gt_folder"]
                ):
        ### Lấy tên các file
        files = None
        if sample_data_txt != None:
            with open(sample_data_txt, "r") as f:
                files = f.read().splitlines()
        else:
            files = os.listdir(train_folder)
        ### Tạo path cho data và label
        self.data_paths = [f"{train_folder}/{file_name}" for file_name in files]
        self.label_paths = [f"{train_gt_folder}/{file_name}" for file_name in files]
        self.preprocess_transform = preprocess_transform
        
    def __len__(self):
        return len(self.data_paths)
    
    def __getitem__(self, index):
        ### Đọc ảnh .jpeg -> PIL
        data_pil = Image.open(self.data_paths[index])
        label_pil = Image.open(self.label_paths[index])
        
        ### Preprocessing ảnh, bao gồm cả việc toTensor
        data = self.preprocess_transform(data_pil)
        label = self.preprocess_transform(label_pil)
        
        ### Normalize:
        data = data/255
        label = label/255
        
        ### Convert label từ ảnh RGB -> ma trận index class {0, 1, 2}
        label = torch.where(label>0.65, 1.0, 0.0)
        label[2, :, :] = 0.0001
        label = torch.argmax(label, 0).type(torch.int64)
        
        return data, label
        

In [ ]:
def split_train_set_val_set(dataset: Dataset, 
                            train_percent: float = config["train_percent"]
                           ):
#     torch.manual_seed(3)
    n_train = int(len(dataset)*train_percent)
    n_val = len(dataset) - n_train
    return random_split(dataset, [n_train, n_val])

In [ ]:
def createDataloader(train_set,
                     val_set,
                     batch_size: int = config["batch_size"]
                    ):
    train_loader= DataLoader(dataset = train_set, batch_size = batch_size, shuffle = True)
    val_loader = DataLoader(dataset = val_set, batch_size = batch_size, shuffle = False)
    return train_loader, val_loader

# **2. Model**

In [ ]:
def createModel():
    return smp.Unet(encoder_name="resnet34",encoder_weights="imagenet",in_channels = 3, classes = 3)

# **3. Hàm Loss**

In [ ]:
def createLoss():
    return nn.CrossEntropyLoss()

# **4. Optimizer và learning rate Scheduler**

In [ ]:
def createOptimizer(model):
    return torch.optim.Adam(params=model.parameters(), lr=config["learning_rate"])

def createLRScheduler(optimizer):
    return torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.6)
    

# **Run**

## Khởi tạo

In [ ]:
# 1. Tạo dataset và dataloader
my_dataset = NeopolypDataset(createPreprocessTransform())
train_set, val_set = split_train_set_val_set(my_dataset)
train_loader, val_loader = createDataloader(train_set, val_set)

In [ ]:
print(train_loader.batch_size)

In [ ]:
# 2. Tạo model
model = createModel().to(device)    

In [ ]:
# 3. Tạo loss
loss_fn = createLoss()

In [ ]:
# 4. Tạo optimizer
optimizer = createOptimizer(model)
lr_scheduler = createLRScheduler(optimizer)

In [ ]:
# Load lại model nếu có
if config["load_model"] != None:
    checkpoint = torch.load(config["load_model"])
    model.load_state_dict(checkpoint["model_state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
    lr_scheduler.load_state_dict(checkpoint["lr_scheduler_state_dict"])

## Train

In [ ]:
def train(train_loader, val_loader,
          model, loss_fn, optimizer, lr_scheduler,
          epochs: int = config["epochs"]
         ):
    
    for epoch in range(1, epochs + 1):
        print(f"Bắt đầu epoch {epoch}")
        t1 = time.time()
        
        # 1. Train
        model.train()
        train_loss = 0
        for batch_data, batch_label in train_loader:
            # 1. predict
            pred = model(batch_data.to(device))
            
            # 2. Tính loss
            loss = loss_fn(pred, batch_label.to(device))
            train_loss += loss.item()
            
            # 3. zero grad
            optimizer.zero_grad()
            
            # 4. Đạo hàm
            loss.backward()
            
            #5. update grad
            optimizer.step()
            
        train_loss /= len(train_loader)
        lr_scheduler.step()
        
        # 2. Val
        val_loss = 0
        model.eval()
        with torch.inference_mode():
            for batch_data, batch_label in val_loader:
                pred = model(batch_data.to(device))
                val_loss += loss_fn(pred, batch_label.to(device))
            
        
        t2 = time.time()
        
        # Print
        print(f"Train Loss: {train_loss}")
        print(f"Val Loss:{val_loss}")
        print(f"Time: {t2 - t1}")
        print(f"End epoch {epoch}\n*****************************")   
        
        # wandb
        wandb.log({"Train Loss": train_loss})
        wandb.log({"Val Loss": val_loss})
               

In [ ]:
train(train_loader, val_loader, model, loss_fn, optimizer, lr_scheduler)

## Infer nộp Test

In [ ]:
class UNetTestDataClass(Dataset):
    def __init__(self,  
                 transform: transforms.Compose,
                 images_path: str = "/kaggle/input/bkai-igh-neopolyp/test/test"):
        super(UNetTestDataClass, self).__init__()
        
        images_list = os.listdir(images_path)
        images_list = [images_path+"/"+i for i in images_list]
        
        self.images_list = images_list
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.images_list[index]
        data = Image.open(img_path)
        h = data.size[1]
        w = data.size[0]
        data = self.transform(data) / 255        
        return data, img_path, h, w
    
    def __len__(self):
        return len(self.images_list)

In [ ]:
test_dataset = UNetTestDataClass(createPreprocessTransform())
test_dataloader = DataLoader(test_dataset,
                         batch_size = config["batch_size"]
                        )

In [ ]:
model.eval()
if not os.path.isdir("/kaggle/working/predicted_masks"):
    os.mkdir("/kaggle/working/predicted_masks")
for _, (img, path, H, W) in enumerate(test_dataloader):
    a = path
    b = img
    h = H
    w = W
    
    with torch.no_grad():
        predicted_mask = model(b.to(device))
    for i in range(len(a)):
        image_id = a[i].split('/')[-1].split('.')[0]
        filename = image_id + ".png"
        mask2img = transforms.Resize((h[i].item(), w[i].item()), interpolation=InterpolationMode.NEAREST)(transforms.ToPILImage()(torch.nn.functional.one_hot(torch.argmax(predicted_mask[i], 0)).permute(2, 0, 1).float()))
        mask2img.save(os.path.join("/kaggle/working/predicted_masks/", filename))

In [ ]:
import numpy as np
import pandas as pd
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 0] = 255
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    return rle_to_string(rle)

def mask2string(dir):
    ## mask --> string
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = '/kaggle/working/predicted_masks' # change this to the path to your output mask folder
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']
df.to_csv(r'output.csv', index=False)

## **SAVE model**

In [ ]:
if not os.path.isdir(config["save_folder"]):
    os.mkdir(config["save_folder"])

checkpoint = {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "lr_scheduler_state_dict": lr_scheduler.state_dict()
}
torch.save(checkpoint, Path(f"{config['save_folder']}/save_model.pth"))